# Settings

In [1]:
import platform  

def get_device():
    """
    Function to determine the device type based on the node name.
    It uses a dictionary to map node names to device types.

    :return: device type as a string
    :raises Exception: if node name is not found in the device_map dictionary
    """
    device_map = {
        "PC-Cristian": "cuda",
        "Dell-G5-15-Alexios": "cuda",
        "MacBook-Pro-di-Cristian.local": "mps",
        "MacBookProDiGrazia": "cpu",
        "DESKTOP-RQVK8SI":"cuda",
        "MacBook-Pro.station":"mps"
    }

    try:
        return device_map[platform.uname().node]
    except KeyError:
        raise Exception("Node name not found. Please add your node name and its corresponding device to the dictionary.")

device = get_device()

In [2]:
import os
from project_paths import *

def delete_ds_store_files(folder):
    for root, dirs, files in os.walk(folder):
        for file in files:
            if file == ".DS_Store":
                file_path = os.path.join(root, file)
                os.remove(file_path)
                print(f"Deleted {file_path}")

# Specifica la cartella principale in cui cercare i file .DS_Store
folder_path = data_folder_path

delete_ds_store_files(folder_path)

# Create YOLO dataset folder

In [3]:
import os, shutil, random

def create_yolo_datset(splitted_frames_path, splitted_gt_path, yolo_dataset_path, num_frames_per_video=10):

    # folders
    # if yolo dataset folder, doesn't exist, create it
    if not os.path.exists(yolo_dataset_path):
        os.makedirs(yolo_dataset_path)  

    # create the yolo dataset structure, we create a folder for train and a folder for val
    if not os.path.exists(os.path.join(yolo_dataset_path, 'train')):
        os.makedirs(os.path.join(yolo_dataset_path, 'train'))
    if not os.path.exists(os.path.join(yolo_dataset_path, 'val')):
        os.makedirs(os.path.join(yolo_dataset_path, 'val'))

    # create the two inner folders: one for fire and one for no fire
    if not os.path.exists(os.path.join(yolo_dataset_path, 'train', 'fire')):
        os.makedirs(os.path.join(yolo_dataset_path, 'train', 'fire'))
    if not os.path.exists(os.path.join(yolo_dataset_path, 'train', 'no_fire')):
        os.makedirs(os.path.join(yolo_dataset_path, 'train', 'no_fire'))
    if not os.path.exists(os.path.join(yolo_dataset_path, 'val', 'fire')):
        os.makedirs(os.path.join(yolo_dataset_path, 'val', 'fire'))
    if not os.path.exists(os.path.join(yolo_dataset_path, 'val', 'no_fire')):
        os.makedirs(os.path.join(yolo_dataset_path, 'val', 'no_fire'))

    # now we copy the images in the right folder
    # we have to copy the images in the train folder and in the val folder
    # for each folder in SPLITTED_FRAMES_DATASET/TRAINING_SET/0 we randomly sample num_frames_per_video frames and put them in dataset/train/no_fire
    # for each folder in SPLITTED_FRAMES_DATASET/VALIDATION_SET/0 we randomly sample num_frames_per_video frames and put them in dataset/val/no_fire

    count = 0
    for folder in os.listdir(os.path.join(splitted_frames_path, 'TRAINING_SET','0')):
        tot_frames = len(os.listdir(os.path.join(splitted_frames_path, 'TRAINING_SET','0')))
        if tot_frames < num_frames_per_video:
            num_frames_per_video = tot_frames
        chosen_frames = []
        for _ in range (num_frames_per_video):
            # randomly sample a frame
            frame = random.choice(os.listdir(os.path.join(splitted_frames_path, 'TRAINING_SET','0', folder)))
            while frame in chosen_frames:
                frame = random.choice(os.listdir(os.path.join(splitted_frames_path, 'TRAINING_SET','0', folder)))
            chosen_frames.append(frame)
            # copy the frame in the right folder
            shutil.copy(os.path.join(splitted_frames_path, 'TRAINING_SET','0', folder, frame), os.path.join(yolo_dataset_path, 'train', 'no_fire', frame))
            # rename the frame
            os.rename(os.path.join(yolo_dataset_path, 'train', 'no_fire', frame), os.path.join(yolo_dataset_path, 'train', 'no_fire', str(count)+'.jpg'))
            count += 1
        
    count = 0
    for folder in os.listdir(os.path.join(splitted_frames_path, 'VALIDATION_SET','0')):
        tot_frames = len(os.listdir(os.path.join(splitted_frames_path, 'VALIDATION_SET','0')))
        if tot_frames < num_frames_per_video:
            num_frames_per_video = tot_frames
        chosen_frames = []
        for _ in range (num_frames_per_video):
            # randomly sample a frame
            frame = random.choice(os.listdir(os.path.join(splitted_frames_path, 'VALIDATION_SET','0', folder)))
            while frame in chosen_frames:
                frame = random.choice(os.listdir(os.path.join(splitted_frames_path, 'VALIDATION_SET','0', folder)))
            chosen_frames.append(frame)
            # copy the frame in the right folder
            shutil.copy(os.path.join(splitted_frames_path, 'VALIDATION_SET','0', folder, frame), os.path.join(yolo_dataset_path, 'val', 'no_fire', frame))
            # rename the frame
            os.rename(os.path.join(yolo_dataset_path, 'val', 'no_fire', frame), os.path.join(yolo_dataset_path, 'val', 'no_fire', str(count)+'.jpg'))
            count += 1

    # # for each folder in SPLITTED_FRAMES_DATASET/TRAINING_SET/1 we randomly sample num_frames_per_video frames starting from the frame indicated in the annotation file
    # # and put them in dataset/train/fire
    # # for each folder in SPLITTED_FRAMES_DATASET/VALIDATION_SET/1 we randomly sample num_frames_per_video frames starting from the frame indicated in the annotation file
    # # and put them in dataset/val/fire

    count = 0    
    num_frames = 0
    num_folders = 0
    for folder in os.listdir(os.path.join(splitted_frames_path, 'TRAINING_SET','1')):
        num_folders += 1
        tot_frames = len(os.listdir(os.path.join(splitted_frames_path, 'TRAINING_SET','1')))
        if tot_frames < num_frames_per_video:
            num_frames_per_video = tot_frames
        chosen_frames = []
        # get the annotation file corresponding to the folder
        annotation_file = folder.replace("mp4", "rtf")
        # open the annotation file
        with open(os.path.join(splitted_gt_path, 'TRAINING_SET', '1', annotation_file), 'r') as f:
            # get the frame in which the fire starts
            start_frame = int(f.readline().split(',')[0])
            # randomly sample a frame whose number starts from start_frame
            available_frames = []
            for _ in range (num_frames_per_video):
                for frame in os.listdir(os.path.join(splitted_frames_path, 'TRAINING_SET','1', folder)):
                    if int(frame.split('.')[0])>=start_frame:
                        available_frames.append(frame)
                frame = random.choice(available_frames)
                while frame in chosen_frames:
                    frame = random.choice(available_frames)
                chosen_frames.append(frame)
                num_frames += 1
                # copy the frame in the right folder
                shutil.copy(os.path.join(splitted_frames_path, 'TRAINING_SET','1', folder, frame), os.path.join(yolo_dataset_path, 'train', 'fire', frame))
                # rename the frame
                os.rename(os.path.join(yolo_dataset_path, 'train', 'fire', frame), os.path.join(yolo_dataset_path, 'train', 'fire', str(count)+'.jpg'))
                count += 1

    count = 0
    for folder in os.listdir(os.path.join(splitted_frames_path, 'VALIDATION_SET','1')):
        tot_frames = len(os.listdir(os.path.join(splitted_frames_path, 'VALIDATION_SET','1')))
        if tot_frames < num_frames_per_video:
            num_frames_per_video = tot_frames
        chosen_frames = []
        # get the annotation file corresponding to the folder
        annotation_file = folder.replace("mp4", "rtf")
        # open the annotation file
        with open(os.path.join(splitted_gt_path, 'VALIDATION_SET', '1', annotation_file), 'r') as f:
            # get the frame in which the fire starts
            start_frame = int(f.readline().split(',')[0])
            # randomly sample a frame whose number starts from start_frame
            available_frames = []
            for _ in range (num_frames_per_video):
                for frame in os.listdir(os.path.join(splitted_frames_path, 'VALIDATION_SET','1', folder)):
                    if int(frame.split('.')[0])>=start_frame:
                        available_frames.append(frame)
                frame = random.choice(available_frames)
                while frame in chosen_frames:
                    frame = random.choice(available_frames)
                chosen_frames.append(frame)
                # copy the frame in the right folder
                shutil.copy(os.path.join(splitted_frames_path, 'VALIDATION_SET','1', folder, frame), os.path.join(yolo_dataset_path, 'val', 'fire', frame))
                # rename the frame
                os.rename(os.path.join(yolo_dataset_path, 'val', 'fire', frame), os.path.join(yolo_dataset_path, 'val', 'fire', str(count)+'.jpg'))
                count += 1

create_yolo_datset(splitted_frames_path, splitted_annotations_path, yolo_folder_path, num_frames_per_video=1)


# Preprocess images

In [4]:
# open all the images in yolo_dataset and resize them to 224x224
from PIL import Image
from tqdm import tqdm
import os

def resize_images(dataset_path, img_size=224):

    folders = ['train', 'val']
    subfolders = ['fire', 'no_fire']

    for folder in folders:
        for subfolder in subfolders:
            for image in tqdm(os.listdir(os.path.join(dataset_path, folder, subfolder))):
                img = Image.open(os.path.join(dataset_path, folder, subfolder, image))
                img = img.resize((img_size, img_size))
                img.save(os.path.join(dataset_path, folder, subfolder, image))

resize_images(yolo_folder_path, 608)

100%|██████████| 14/14 [00:00<00:00, 96.21it/s]


# Train YOLO

In [5]:
from ultralytics import YOLO

# Load a model
model = YOLO("yolov8n-cls.pt")  # load a pretained model

model.train(data=yolo_folder_path, imgsz=608, epochs=50, rect=True, device=device)  # train the model

Ultralytics YOLOv8.0.135 🚀 Python-3.11.4 torch-2.0.1 MPS (Apple M1 Pro)
yolo/engine/trainer: task=classify, mode=train, model=yolov8n-cls.pt, data=/Users/graziaferrara/Desktop/MachineLearning/yolo_dataset, epochs=50, patience=50, batch=16, imgsz=608, save=True, save_period=-1, cache=False, device=mps, workers=8, project=None, name=None, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=True, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, line_width=None, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, boxes=True, format=torchscript, keras=False, optimize=False,

# Test YOLO

In [8]:

# Evaluate a trained model
yolo_weights_path = "../weights/best.pt"
test_videos_folder = "../test_data/TEST_SET"
test_frames_folder = "../yolo_dataset/test"

# create the test frames folder if it doesn't exist
if not os.path.exists(test_frames_folder):
    os.mkdir(test_frames_folder)

model = YOLO(yolo_weights_path)  # load a trained model



In [10]:
import cv2, os, glob, tqdm
import pandas as pd

# create dataframe to which save if a clip is fire or not and the discovering frame
df = pd.DataFrame(columns=['video', 'fire', 'frame'])

for video in tqdm.tqdm(glob.glob(os.path.join(test_videos_folder, "**"), recursive=True)):
    if os.path.isdir(video):
        continue
    # Process the video
    ret = True
    cap = cv2.VideoCapture(video) # Decodifica lo streaming
    fps = cap.get(cv2.CAP_PROP_FPS) # Ottiene il frame rate
    f = 0
    fire = 0
    while ret:
        ret, img = cap.read() # Chiamando read leggiamo il frame successivo dallo stream
        if ret: # ret è false quando non ci sono più frame da leggere
            f += 1
            # Il tensore img letto viene trasformato tramite la classe PIL e lo salviamo
            frame_name = os.path.join(test_frames_folder, "{:05d}.jpg".format(f))
            
            cv2.resize(img, (608, 608))
            cv2.imwrite(frame_name, img)
            # evaluate YOLO model on the frame
            results = model.predict(frame_name, verbose=False)
            # if it is a fire tell the video is fire and continue with the next video
            for result in results:
                if result.probs.data.tolist()[0] >= 0.5:
                    fire = 1
                    ret = 0
                    break
    # save the result in the dataframe
    df.loc[len(df)] = [video, fire, round(f/fps)]
    #empty the test frames folder
    # for frame in os.listdir(test_frames_folder):
    #     os.remove(os.path.join(test_frames_folder, frame))
    cap.release()

        

100%|██████████| 148/148 [02:30<00:00,  1.01s/it]


In [12]:
annotations_path = '../test_data/GT_TEST_SET'
# modify the first column of the dataframe to remove the path and keep only the video name
df['video'] = df['video'].apply(lambda x: x.split('/')[-1])
# add a column with the GT taken from test_data/GT_TEST_SET
df['GT'] = None
# for each column of the dataframe, open the annotation, take the umber and paste it in the GT column of the dataframe
for i in range(len(df)):
    # open the annotation file
    annotation_file = df['video'][i].replace("mp4", "rtf")
    with open(os.path.join(annotations_path, annotation_file), 'r') as f:
        line = f.readline()
        if line:
            start_frame = int(line.split(',')[0])
            df['GT'][i] = start_frame
        else:
            df['GT'][i] = None


/var/folders/98/yrs1tbqs4mn9v5k3tpqcr4_m0000gn/T/ipykernel_52095/2130210895.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['GT'][i] = start_frame
/var/folders/98/yrs1tbqs4mn9v5k3tpqcr4_m0000gn/T/ipykernel_52095/2130210895.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['GT'][i] = start_frame
/var/folders/98/yrs1tbqs4mn9v5k3tpqcr4_m0000gn/T/ipykernel_52095/2130210895.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  

In [13]:
from sklearn.metrics import precision_score, recall_score

Y = []
Y_hat = []
video_counter = 0
guard_time = 5 # seconds
delays = []

for i in range(len(df)):
    g_frame = df['GT'][i]
    p_frame = df['frame'][i]
    if g_frame != None:
        # There is a fire in the video
        Y.append(1)
        if p_frame >= max(0, g_frame - guard_time):
            # Detection is fast enough
            Y_hat.append(1)
            delays.append(abs(p_frame - g_frame))
        else:
            # Detection is not fast enough
            Y_hat.append(0)
    else:
        # No fire in video
        Y.append(0)
        Y_hat.append(df['fire'][i])

accuracy = sum([1 for i in range(len(Y)) if Y[i] == Y_hat[i]])/len(Y)
precision = precision_score(Y, Y_hat)
recall = recall_score(Y, Y_hat)

if len(delays):
    D = sum(delays)/len(delays) 
else:
    print("Can't calculate D because no fire detected in the test set")
Dn = max(0, 60-D)/60

print("Accuracy: {:.4f}".format(accuracy))
print("Precision: {:.4f}".format(precision))
print("Recall: {:.4f}".format(recall))
print("F-score: {:.4f}".format(2 * precision * recall / (1e-10 + precision + recall)))
print("Detection mean error: {:.4f}".format(D))
print("Detection delay: {:.4f}".format(Dn))
    
print("Final score numerator: {:.4f}".format(precision * recall * Dn))

Accuracy: 0.8980
Precision: 0.8980
Recall: 1.0000
F-score: 0.9462
Detection mean error: 0.7197
Detection delay: 0.9880
Final score numerator: 0.8872


In [ ]:
# save the dataframe in a csv file
df.to_csv('yolo_results.csv', index=False)